## Introduction
Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. If you're inspired to dig deeper, click the blue "Fork Notebook" button at the top of this kernel to begin editing.

## Exploratory Analysis
To begin this exploratory analysis, first import libraries and define functions for plotting the data using `matplotlib`. Depending on the data, not all plots will be made. (Hey, I'm just a simple kerneling bot, not a Kaggle Competitions Grandmaster!)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

There is 0 csv file in the current version of the dataset:


In [ ]:
from scipy.io import loadmat 
import pandas as pd
data = loadmat(r"/kaggle/input/AlphaNumaric.mat")
# data  = {k:v for k, v in data.items() if k[0] != '_'}
# df = pd.DataFrame({k: pd.Series(v[0]) for k, v in data.items()})  
# df.to_csv("example.csv")

The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [ ]:
data  = {k:v for k, v in data.items() if k[0] != '_'}

In [ ]:
df_y = pd.DataFrame(data['y']).T
df_X = pd.DataFrame(data['X']).T

In [ ]:
letters = [chr(i) for i in range(ord('0'), ord('9')+1)] + [chr(i) for i in range(ord('a'), ord('z')+1)]
letters = letters[:36]  # Ensure it matches the number of classes

# Generate column names for X based on letter labels
feature_names = []
for letter in letters:
    feature_names.extend([f'{letter}_Feature_{i+1}' for i in range(100)])

# Ensure we have the correct number of feature names
if len(feature_names) != df_X.shape[1]:
    raise ValueError("Feature names length does not match the number of columns in X_transposed.")

# Set column names for X
df_X.columns = feature_names

class_column_names = []
for letter in letters:
    class_column_names.extend([f'{letter}_class_{i+1}' for i in range(100)])

# Ensure we have the correct number of class names
if len(class_column_names) != df_y.shape[1]:
    raise ValueError("Class names length does not match the number of columns in df_y.")

# Set column names for y
df_y.columns = class_column_names

Oh, no! There are no automatic insights available for the file types used in this dataset. As your Kaggle kerneler bot, I'll keep working to fine-tune my hyper-parameters. In the meantime, please feel free to try a different dataset.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
print("Shape of X:", df_X.shape)
print("Shape of y:", df_y.shape)

In [ ]:
# Transpose the DataFrames to match the correct shapes
df_X = df_X.T
df_y = df_y.T

# Verify the new shapes
print("Corrected Shape of X:", df_X.shape)  # Should be (3600, 400)
print("Corrected Shape of y:", df_y.shape)  # Should be (3600, 36)

In [ ]:
# Convert DataFrames to numpy arrays
X_values = df_X.values
y_values = df_y.values


In [ ]:
# Assuming y_values is in one-hot encoded format already
# If not, you should convert it to one-hot encoded format using to_categorical()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_values, y_values, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(36, activation='softmax')  # 36 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train,
                    epochs=20,  # Adjust number of epochs as needed
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc:.4f}')

# Optionally, save the model
model.save('letter_recognition_model.h5')


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os


# Parameters
img_size = (20, 20)  # Assuming each image is 20x20 pixels
num_classes = 36

# Create directories to save images
os.makedirs('images', exist_ok=True)
os.makedirs('labels', exist_ok=True)

# Convert each row in X to an image and save
for i in range(X_values.shape[0]):
    img_array = X_values[i].reshape(img_size[0], img_size[1])  # Reshape to 2D
    img = Image.fromarray(np.uint8(img_array * 255))  # Convert to image, scaling feature values to 0-255
    img.save(f'images/image_{i}.png')

# Save labels to a file
np.save('labels.npy', y_values)  # Save labels as numpy array for later use


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Path to the dataset folder
dataset_path = '/kaggle/working/images/'

# List files in the dataset directory
files = os.listdir(dataset_path)

# Select a few images to display
num_images_to_display = 169
plt.figure(figsize=(14, 14))

# Set number of rows and columns for the grid
rows = 13
cols = 13

for i in range(min(num_images_to_display, len(files))):
    img_path = os.path.join(dataset_path, files[i])
    img = Image.open(img_path)
    img = img.transpose(Image.FLIP_LEFT_RIGHT) # Mirror the image horizontally
    img = img.rotate(90, expand=True)
    
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.title(f"Image {i+1}", fontsize=10)  # Smaller font size for titles
    plt.axis('off')  # Hide axes

plt.tight_layout(pad=0.2)  # Adjust subplots to fit into the figure area with some padding

# Save the figure as a PNG file
output_file_path = '/kaggle/working/collage.png'
plt.savefig(output_file_path, format='png')

# Show the figure
plt.show()

print(f"Collage saved as {output_file_path}")

In [ ]:
import os

# List files in the working directory
print(os.listdir('/kaggle/working'))

## Conclusion
This concludes your starter analysis! To go forward from here, click the blue "Fork Notebook" button at the top of this kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!